# BERT

## Requirements

In [ ]:
pip install -r requirements.txt

In [2]:
# Imports
import numpy as np
import pandas as pd
import torch
import transformers as ppb  # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

from statistics import mean

import _helpers as hp

## Load Data
For chosing which ASR data change the variable by the key in the paths dictionary

In [4]:
# Defining paths for our data. "ASR" means the TDNNF-LFMMI method, and "new_ASR" means the Wave2vec method.
paths = {
    "ground_truth": "snips/merged_GT_data.csv",
    "ASR": "snips/ASR_with_labels.csv",
    "new_ASR": "snips/new_ASR_with_labels.csv",
    "new_ASR_Autocorrect": "snips/new_ASR_Autocorrection_with_labels.csv",
}

# Chosing which ASR data we will use
ASR_data = pd.read_csv(paths["new_ASR"])

# Groundtruth data
GT_data = pd.read_csv(paths["ground_truth"])

## Import pre-trained DistilBERT model and tokenizer

DistilBERT is a small, fast, cheap and light Transformer model. It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of BERT's performances

In [94]:
model_class, tokenizer_class, pretrained_weights = (
    ppb.DistilBertModel,
    ppb.DistilBertTokenizer,
    "distilbert-base-uncased",
)

## Import pre-trained BERT model and tokenizer

If we want to use BERT instead of DistilBERT we comment the previous model importation and use this one instead:

In [95]:
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-small-uncased')

In [96]:
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Tokenization

In [97]:
tokenized_GT_data = GT_data["transcript"].apply(
    (lambda x: tokenizer.encode(x, add_special_tokens=True))
)

tokenized_ASR_data = ASR_data["transcript"].apply(
    (lambda x: tokenizer.encode(x, add_special_tokens=True))
)

The problem now is that the tokenized vectors ar not with the name size

In [98]:
if max(tokenized_GT_data.apply(len)) > max(tokenized_ASR_data.apply(len)):
    _max = max(tokenized_GT_data.apply(len))
    _min = min(tokenized_GT_data.apply(len))
else:
    _max = max(tokenized_ASR_data.apply(len))
    _min = min(tokenized_ASR_data.apply(len))

print("MX:", _max)
print("MIN:", _min)

MX: 22
MIN: 3


lets fix that with a simple padding function

In [99]:
padded_GT_data = np.array(
    list(tokenized_GT_data.apply(lambda x: hp.padding_func(x, _max)))
)
padded_ASR_data = np.array(
    list(tokenized_ASR_data.apply(lambda x: hp.padding_func(x, _max)))
)
print(padded_GT_data.shape)
print(padded_ASR_data.shape)

(2224, 22)
(1660, 22)


# Masking

If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [100]:
attention_mask_GT = np.where(padded_GT_data != 0, 1, 0)

attention_mask_ASR = np.where(padded_ASR_data != 0, 1, 0)

# Processing with DistrilBert

In [101]:
input_ids_GT = torch.tensor(padded_GT_data)
attention_mask_GT = torch.tensor(attention_mask_GT)

with torch.no_grad():
    last_hidden_states_GT = model(input_ids_GT, attention_mask=attention_mask_GT)

In [102]:
input_ids_ASR = torch.tensor(padded_ASR_data)
attention_mask_ASR = torch.tensor(attention_mask_ASR)

with torch.no_grad():
    last_hidden_states_ASR = model(input_ids_ASR, attention_mask=attention_mask_ASR)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called [CLS] (for classification) at the beginning of every sentence

basically [CLS] contains all information of the sentence and representing the sentence-level classification.

In [103]:
# Taking the CLS features of each sentence
features_GT = last_hidden_states_GT[0][:, 0, :].numpy()
features_GT.shape

(2224, 768)

In [104]:
# Taking the CLS features of each sentence
features_ASR = last_hidden_states_ASR[0][:, 0, :].numpy()
features_ASR.shape

(1660, 768)

In [105]:
labels_GT = GT_data["user_action_num"]
labels_ASR = ASR_data["user_action_num"]

# Classifiers

## Train/Test Split

The function beneath provides the features and labels needed for testing. Using the loaded ASR or not (then using ground truth data) is decided by input. As standard we use the ASR dataset.

In [106]:
def get_train_test_data(type_of_dataset="ASR", train_size=0.9):
    """Retrieves the relevant dataset and splits according to parameter"""
    # If ASR, give ASR features and labels
    if type_of_dataset == "ASR":
        train_features, test_features, train_labels, test_labels = train_test_split(
            features_ASR, labels_ASR, train_size=train_size
        )
    # If the dataset is not the ASR data, use the ground truth data
    else:
        train_features, test_features, train_labels, test_labels = train_test_split(
            features_GT, labels_GT, train_size=train_size
        )

    return train_features, test_features, train_labels, test_labels


def run_classifier(
    classifier_pipe, type_of_dataset="ASR", train_size=0.9, number_of_times=100
):
    """For running the classifiers multiple times, and returning mean accuracy score. Wraps around get_train_test_data"""
    mean_score_list = []
    n = number_of_times
    for i in range(n):
        train_features, test_features, train_labels, test_labels = get_train_test_data(
            type_of_dataset="ASR"
        )
        classifier_pipe.fit(train_features, train_labels)

        classifier_pred_labels = classifier_pipe.predict(test_features)  # predictions

        classifier_score = classifier_pipe.score(test_features, test_labels)  # accuracy

        mean_score_list.append(lgr_score)
    return mean_score, classifier_pred_labels, classifier_score

## Logistic Regression

In [107]:
lgr = LogisticRegression(
    C=0.6, max_iter=1000, penalty="l2", solver="liblinear"
)  # Create the classification model

lgr_pipe = make_pipeline(preprocessing.StandardScaler(), lgr)  # Scale feature space

mean_score, lgr_pred_labels, lgr_score = run_classifier(
    classifier_pipe=lgr_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.728


### Model Evaluation

In [108]:
print(
    classification_report(
        test_labels,
        lgr_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, lgr_pred_labels))

print("\nACCURACY:", lgr_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.12      0.12      0.12        24
     SwitchLightOn       0.11      0.11      0.11        28
IncreaseBrightness       0.10      0.10      0.10        30
DecreaseBrightness       0.05      0.04      0.04        24
SetLightBrightness       0.21      0.22      0.21        32
     SetLightColor       0.21      0.21      0.21        28

          accuracy                           0.14       166
         macro avg       0.13      0.13      0.13       166
      weighted avg       0.14      0.14      0.14       166

[[3 2 9 0 7 3]
 [4 3 6 5 4 6]
 [4 6 3 5 7 5]
 [2 4 6 1 6 5]
 [8 7 1 5 7 4]
 [3 5 5 6 3 6]]

ACCURACY: 0.7951807228915663


# Naive Bayes

### Gaussian Naive Bayes

In [109]:
gnb = GaussianNB()  # Create the classification model

gnb_pipe = make_pipeline(preprocessing.StandardScaler(), gnb)  # Scale feature space

mean_score, gnb_pred_labels, gnb_score = run_classifier(
    classifier_pipe=gnb_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.728


### Model Evaluation

In [110]:
print(
    classification_report(
        test_labels,
        gnb_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, gnb_pred_labels))

print("\nACCURACY:", gnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.17      0.21      0.19        24
     SwitchLightOn       0.19      0.32      0.24        28
IncreaseBrightness       0.15      0.07      0.09        30
DecreaseBrightness       0.05      0.04      0.05        24
SetLightBrightness       0.22      0.22      0.22        32
     SetLightColor       0.21      0.18      0.19        28

          accuracy                           0.17       166
         macro avg       0.16      0.17      0.16       166
      weighted avg       0.17      0.17      0.16       166

[[ 5  5  3  2  6  3]
 [ 3  9  0  6  6  4]
 [ 4 11  2  5  5  3]
 [ 5 10  2  1  4  2]
 [ 3  8  5  2  7  7]
 [10  5  1  3  4  5]]

ACCURACY: 0.5180722891566265


## SVM

In [111]:
svm = SVC()  # Create the classification model

svm_pipe = make_pipeline(preprocessing.StandardScaler(), svm)  # Scale feature space

mean_score_list = []

mean_score, svm_pred_labels, svm_score = run_classifier(
    classifier_pipe=svm_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.728


### Model Evaluation

In [112]:
print(
    classification_report(
        test_labels,
        svm_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, svm_pred_labels))

print("\nACCURACY:", svm_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.11      0.17      0.14        24
     SwitchLightOn       0.26      0.25      0.25        28
IncreaseBrightness       0.15      0.10      0.12        30
DecreaseBrightness       0.19      0.25      0.21        24
SetLightBrightness       0.17      0.12      0.15        32
     SetLightColor       0.10      0.11      0.11        28

          accuracy                           0.16       166
         macro avg       0.16      0.17      0.16       166
      weighted avg       0.17      0.16      0.16       166

[[4 4 2 5 4 5]
 [5 7 5 3 3 5]
 [8 5 3 3 4 7]
 [8 1 2 6 2 5]
 [5 6 6 7 4 4]
 [5 4 2 8 6 3]]

ACCURACY: 0.7048192771084337


## NEURAL NETWORK

In [113]:
mlp = MLPClassifier(
    hidden_layer_sizes=(100, 100, 100), activation="relu", solver="adam", max_iter=5000
)  # Create the classification model

mlp_pipe = make_pipeline(preprocessing.StandardScaler(), mlp)  # Scale feature space

mean_score, mlp_pred_labels, mlp_score = run_classifier(
    classifier_pipe=mlp_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.728


### Model Evaluation

In [114]:
print(
    classification_report(
        test_labels,
        mlp_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, mlp_pred_labels))

print("\nACCURACY:", mlp_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.27      0.25      0.26        24
     SwitchLightOn       0.35      0.21      0.27        28
IncreaseBrightness       0.22      0.20      0.21        30
DecreaseBrightness       0.20      0.25      0.22        24
SetLightBrightness       0.25      0.25      0.25        32
     SetLightColor       0.13      0.18      0.15        28

          accuracy                           0.22       166
         macro avg       0.24      0.22      0.23       166
      weighted avg       0.24      0.22      0.23       166

[[6 3 3 5 3 4]
 [5 6 5 3 3 6]
 [2 0 6 6 8 8]
 [4 3 3 6 2 6]
 [3 2 4 6 8 9]
 [2 3 6 4 8 5]]

ACCURACY: 0.7590361445783133


## Try your self

In [115]:
def user_friendly(sentence, mdl):
    """return action from sentence"""
    sent_token = tokenizer.encode(sentence, add_special_tokens=True)
    sent_pad = np.array([sent_token + [0] * (_max - len(sent_token))])
    sent_att_mask = np.where(sent_pad != 0, 1, 0)
    sent_input_ids = torch.tensor(sent_pad)
    sent_attention_mask = torch.tensor(sent_att_mask)

    with torch.no_grad():
        sent_last_hidden_states = model(
            sent_input_ids, attention_mask=sent_attention_mask
        )

    feature = sent_last_hidden_states[0][:, 0, :].numpy()

    prediction = mdl.predict(feature)
    user_action = hp.indx2action(prediction)
    return user_action

In [116]:
user_friendly("red light to the room", lgr_pipe)

['SetLightColor']